In [4]:
import dtale
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import plotly.express as px


In [5]:
df = pd.read_csv("games.csv")

In [7]:
dtale.show(df)

In [25]:
# Filtering
df1 = df[df["rated"]==True].drop(columns=["rated","white_id","black_id"])

In [29]:
df1.describe()

,created_at,last_move_at,turns,white_rating,black_rating,opening_ply
count,1.615500e+04,1.615500e+04,16155.000000,16155.000000,16155.000000,16155.000000
mean,1.483600e+12,1.483601e+12,61.962550,1597.102012,1593.503188,4.814361
std,2.857839e+10,2.857839e+10,33.732775,284.178397,289.627160,2.767543
min,1.376772e+12,1.376772e+12,1.000000,784.000000,789.000000,1.000000
25%,1.477393e+12,1.477394e+12,39.000000,1396.000000,1390.000000,3.000000
50%,1.496360e+12,1.496360e+12,57.000000,1577.000000,1573.000000,4.000000
75%,1.503260e+12,1.503260e+12,80.000000,1794.000000,1791.000000,6.000000
max,1.504493e+12,1.504494e+12,349.000000,2622.000000,2588.000000,28.000000


Executing shutdown due to inactivity...


2021-03-29 00:37:08,792 - INFO     - Executing shutdown due to inactivity...


Executing shutdown...


2021-03-29 00:37:12,853 - INFO     - Executing shutdown...
